In [1]:
import requests
import pandas as pd  

In [2]:
#MI Annual Plant Data to get Plant Codes
api = '' #Register for an API key here https://www.eia.gov/opendata/register.php

#data features to use
data = ['total-consumption','total-consumption-btu','generation','gross-generation',
        'consumption-for-eg','consumption-for-eg-btu','average-heat-content']


def api_url (data=['total-consumption'], fuel=['WH','NG'], states=[], plant=[],
             frequency='monthly', start='', end='', api=api):
    
    datas = ''.join([f'data[]={x}&' for x in data])
    fuels = ''.join([f'facets[fuel2002][]={x}&' for x in fuel])
    plants = ''.join([f'facets[plantCode][]={x}&' for x in plant])
    statess = ''.join([f'facets[state][]={x}&' for x in states])
    freq = f'frequency={frequency}&'
    api_key = f'api_key={api}'

    url = f'https://api.eia.gov/v2/electricity/facility-fuel/data?{datas}{fuels}{statess}{plants}{freq}{api_key}'
    
    return url

In [3]:
def pd_from_api (data=['total-consumption'], fuel=['WH','NG'], states=[], plants=[],
             frequency='monthly', start='', end='', api=api):
    if plants==[]:
        url = api_url(data=data, fuel=fuel, states=states,
             frequency=frequency, start=start, end=end, api=api)
        r = requests.get(url)
        txt = r.json()
        df = pd.DataFrame(txt['response']['data'])
    else:
        for code in plants:
            url = api_url(data=data, fuel=fuel, states=states, plant=[code],
                 frequency=frequency, start=start, end=end, api=api)
            r = requests.get(url)
            txt = r.json()
            temp_df = pd.DataFrame(txt['response']['data'])
            try:
                df = pd.concat([temp_df, df], ignore_index=True, axis=0)
            except:
                df = temp_df
            
    df['plantCode'] = df['plantCode'].astype(int) #so can merge with location data Plant_Code
    
    return df


In [4]:
#Finding plant codes for MI
MI_plants = pd_from_api(states=['MI'],frequency='annual')
len(MI_plants.plantCode.unique())

129

In [5]:
#Using MI plant codes to create csv of all the MI plant data
MI_df = pd_from_api(data=data, states=['MI'],frequency='monthly', plants=MI_plants.plantCode.unique())
MI_df.to_csv('MI_Plant.csv', index=False)

In [6]:
MI_df = pd.read_csv('MI_Plant.csv')
MI_df.head(2)

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,total-consumption,total-consumption-units,...,gross-generation,consumption-for-eg,consumption-for-eg-units,consumption-for-eg-btu,average-heat-content,average-heat-content-units,total-consumption-btu-units,generation-units,gross-generation-units,consumption-for-eg-btu-units
0,2023-10,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,547826.0,MMBtu per Mcf,...,75801.0,547826.0,Mcf,576313.0,1.052,MMBtu per Mcf,MMBtu,megawatthours,megawatthours,MMBtu
1,2023-10,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,CA,0.0,MMBtu per Mcf,...,20056.0,0.0,Mcf,0.0,0.000,MMBtu per Mcf,MMBtu,megawatthours,megawatthours,MMBtu


In [7]:
location_df = pd.read_csv('Power_Plants_in_the_U.S..csv')
location_df.head()

,X,Y,FID,OBJECTID,Plant_Code,Plant_Name,Utility_ID,Utility_Na,sector_nam,Street_Add,...,NG_MW,Nuclear_MW,Crude_MW,Solar_MW,Wind_MW,Other_MW,Source,Period,Longitude,Latitude
0,-89.6850,45.1786,1,13170,4068,Merrill,20860,Wisconsin Public Service Corp,Electric Utility,306 S Park St,...,0.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305,-89.6850,45.1786
1,-87.7586,45.1136,2,13171,4071,Potato Rapids,20860,Wisconsin Public Service Corp,Electric Utility,N4295 Potato Rapids Rd,...,0.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305,-87.7586,45.1136
2,-88.0086,44.5400,3,13172,4072,Pulliam,20860,Wisconsin Public Service Corp,Electric Utility,1530 N Bylsby Av,...,79.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305,-88.0086,44.5400
3,-88.0678,45.2333,4,13173,4074,Sandstone Rapids,20860,Wisconsin Public Service Corp,Electric Utility,N7633 Sandstone Lane,...,0.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305,-88.0678,45.2333
4,-89.7306,45.4411,5,13174,4075,Tomahawk,20860,Wisconsin Public Service Corp,Electric Utility,W6080 Pride Pond Rd,...,0.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305,-89.7306,45.4411


In [8]:
#Some MI plant codes do not appear in location df... will try to find more location data 
location_df.loc[location_df['Plant_Code'] == 52017]

,X,Y,FID,OBJECTID,Plant_Code,Plant_Name,Utility_ID,Utility_Na,sector_nam,Street_Add,...,NG_MW,Nuclear_MW,Crude_MW,Solar_MW,Wind_MW,Other_MW,Source,Period,Longitude,Latitude


In [9]:
combo_df = pd.merge(MI_df, location_df, how='left', left_on='plantCode', right_on='Plant_Code')
combo_df.to_csv('MI_Plant_Location.csv', index=False)

In [10]:
combo_df.sample(5)

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,total-consumption,total-consumption-units,...,NG_MW,Nuclear_MW,Crude_MW,Solar_MW,Wind_MW,Other_MW,Source,Period,Longitude,Latitude
28072,2005-06,10819,Ada Cogeneration LP,NG,Natural Gas,MI,Michigan,CA,24957.79,MMBtu per Mcf,...,30.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-85.494071,42.962672
48806,2006-10,7806,Pine Street,NG,Natural Gas,MI,Michigan,IC,18.77,MMBtu per Mcf,...,7.7,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-83.449953,43.740164
35811,2001-05,10208,Mead Paper,NG,Natural Gas,MI,Michigan,,231516.00,MMBtu per Mcf,...,0.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-87.089100,45.804400
35544,2018-02,10208,Escanaba Mill,NG,Natural Gas,MI,Michigan,ST,607210.00,MMBtu per Mcf,...,0.0,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-87.089100,45.804400
15047,2001-11,1734,Northeast,NG,Natural Gas,MI,Michigan,,0.00,MMBtu per Mcf,...,77.0,0.0,39.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-83.038100,42.450000
